<a href="https://colab.research.google.com/github/MidoriTinto/Python_For_Data_Analysis/blob/main/4_Wrangle_the_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Merging dataframes, creating new columns, replacing with default or correctly formatted values
---

## Merging dataframes

We often want to use data from a set of different data files, or a set of dataframes we have prepared, to combine into one dataframe.

### To merge two dataframes together when both dataframes have column headings in common - stacking them one on top of another 

If `df1` and `df2` have some column headings in common, to combine the two into one dataframe we use:  

`pd.concat([dataframes to combine])`

This will create a new dataframe with all columns from the original two dataframes, which we can store in a new variable for later use.  Missing values are filled with null values. 

If we want only the column headings that appear in both tables, we can use join='inner':

`pd.concat([dataframes to combine], join='inner')` 

We may also need to refactor the indexing, where two tables are indexed from 0 upwards, adding the two tables together will result in multiple occurences of the same index.  This can be overcome by using ignore_index=True:

`pd.concat([dataframes to combine], join='inner', ignore_index=True)`


### Merging dataframes by common column matching values

 `pd.merge(datframe1, dataframe2)`

 extra parameters: 
 * which columns to merge by when the column name is the same in both dataframes: 
  * single column `pd.merge(df, df2, on='column_name')`  
  * multiple columns `pd.merge(df, df2, on=['column1', 'column2']`

* which columns to merge by when columns have different names in each dataframe 
  * single column `pd.merge(df1, df2, left_on = 'df1_column', right_on = 'df2_column')`
  * multiple column `pd.merge(df1, df2, left_on = ['df1_column1', 'df1_column2'], right_on = ['df2_column1', 'df2_column2'])`
* what kind of join (inner, left, right or cross) inner is default
  * `pd.merge(df1, df2, how = '....')`
  * inner will join with all columns
  * left joins with just first stated df columns 
  * right joins with just second stated df columns
  * cross is a mix of both

* To specify only specific columns to be returned from each dataframe
  * `pd.merge(df1[['column1', 'column2']], df2[['column1', 'column2', 'column3']], on = ['column1', 'column2'])
  * you must include the columns from each dataframe that are being merged on



### Difference between concat and merge: 

```
df1:  
          Key  data1
      0   b   0
      1   b   1
      2   a   2
      3   c   3
      4   a   4
      5   a   5
      6   b   6

df2:
    Key data2
0   a   0
1   b   1
2   d   2

# merge would look like this:

pd.merge(df1, df2)

   Key data1 data2
0   b    0    1
1   b    1    1
2   b    6    1
3   a    2    0
4   a    4    0
5   a    5    0

# concat would look like this

pd.concat([df1, df2])

   Key data1 data2
0   b   0     NaN
1   b   1     NaN
2   a   2     NaN
3   c   3     NaN
4   a   4     NaN
5   a   5     NaN
6   b   6     NaN
0   a   Nan   0
1   b   Nan   1
2   d   Nan   2
```

Merge is particularly useful with datasets that share a unique index eg. two datasets that were indexed by date or country 


### Exercise 1 - combine the two data sets

The Excel file at this URL https://github.com/futureCodersSE/working-with-data/blob/main/Data%20sets/Income-Data.xlsx?raw=true contains TWO data sheets named county-level and state-level.  

Read the county-level sheet into a dataframe called **county_level_df** 
Read the state-level sheet into a dataframe called **state_level_df** 

Write a function called **combine_whole** which: 

Uses `pd.concat([list of dataframes])` to combine the two dataframes into a new dataframe called **income_df**, filling missing values with null values.  


In [ ]:
import pandas as pd
url="https://github.com/futureCodersSE/working-with-data/blob/main/Data%20sets/Income-Data.xlsx?raw=true"
income=pd.read_excel(url)
income

county_level_df = pd.read_excel(url, sheet_name="county-level")
state_level_df = pd.read_excel(url, sheet_name= "state-level")

def combine_whole():
  # add code to combine the 2 dataframes, filling missing values with NA
  df_list=[county_level_df, state_level_df]
  income_df=pd.concat(df_list)
  return income_df


# create new dataframe from your function above 
income_df = combine_whole()

# This will run and test your code to check your new dataframe contains null values 
actual = income_df.isnull().values.any()
expected = True 

if actual == expected:
  print("Test passed", actual)
else:
  print("Test failed, expected", expected, "got", actual)

combine_whole()

Test passed True


,State,County,Population,Age,Income,Pop,Education
0,TX,1.0,72.0,34,65,NaN,NaN
1,TX,2.0,33.0,42,45,NaN,NaN
2,TX,5.0,25.0,23,46,NaN,NaN
3,TX,6.0,54.0,36,65,NaN,NaN
4,TX,7.0,11.0,42,53,NaN,NaN
5,TX,8.0,28.0,25,62,NaN,NaN
6,TX,9.0,82.0,35,66,NaN,NaN
7,TX,10.0,5.0,40,75,NaN,NaN
8,MD,11.0,61.0,27,22,NaN,NaN
9,MD,2.0,5.0,23,69,NaN,NaN


### Exercise 2 - ignoring index to get a new indexing system
---

All rows in each dataframe are indexed from 0 to one less than the number of rows.  You may have noticed that the concatenation in the previous exercise has kept the indexing from the individual tables.  

If we are making a new table it may make sense to create a new set of indices, from 0 to one less than the length of the new table.   Do this by adding an extra paramater ignore_index=True.  ignore_index is False by default and all original indices are kept.

We can also use `join='inner'` to only join columns which are common to both tables

Write a function called **combine_common** which will: 

Combine the dataframes `county_level_df` and `state_level_df` into a new dataframe called `income_df`, adding the parameter `ignore_index=True` and join via inner join. 


In [ ]:
def combine_common():
  # add code to combine the 2 dataframes ignoring index and using inner join
  df_list=[county_level_df, state_level_df]
  income_df=pd.concat((df_list), join='inner', ignore_index=True)
  return income_df

# create new dataframe from your function above 
income_df = combine_common()


# This will run and test your code to check your new dataframe ends with the correct index and has the right number of columns 
actual = income_df.index[-1]
expected = 22

if actual == expected and len(income_df.columns) == 3:
  print("Test passed", actual)
else:
  print("Test failed, expected last row index of", expected, "got", actual, "and expected 3 columns but got", len(income_df.columns))

combine_common()

Test passed 22


,State,Age,Income
0,TX,34,65
1,TX,42,45
2,TX,23,46
3,TX,36,65
4,TX,42,53
5,TX,25,62
6,TX,35,66
7,TX,40,75
8,MD,27,22
9,MD,23,69


# Appending rows to a dataframe

Where two dataframes have matching columns, we can append one to the other to add the records from one onto the end of the other.

We do this using dataframe.append()

`income_2 = income.append()`

### Exercise 3 - add new rows to the end of the income dataframe

The sheet `income` in the Excel data file has 10 further records showing State, Age and Income only, so this table matches the income dataframe exactly.

Read the data from sheet_name `income` in the same Excel data file into a new dataframe called **income_new**.  
 
Write a function called **combine_income** which will: 

Append this dataframe to the `income_df` dataframe to form a new dataframe called **income_df_v2**.  Use the ignore_index=True parameter to reindex.  


In [ ]:
import pandas as pd
url="https://github.com/futureCodersSE/working-with-data/blob/main/Data%20sets/Income-Data.xlsx?raw=true"
income_new = pd.read_excel(url, sheet_name="income")


def combine_income():
  # add code below to join the income_new dataframe with the income_df dataframe ignoring the index
  income_df_V2 = income_new.append(income_df, ignore_index=True)
  return income_df_V2

# save returned dataframe in a variable
income_df_v2 = combine_income()
print(income_df_v2)

# This will run and test your code to see if new dataframe has correct number of columns and correct indexing 
actual = income_df_v2.index[-1]
expected = 32

if actual == expected and len(income_df_v2.columns) == 4:
  print("Test passed", actual)
else:
  print("Test failed, expected last row index of", expected, "got", actual, "and expected 4 columns but got", len(income_df_v2.columns))


   State  Age  Income   Pop
0     TX   32      69  70.0
1     MD   41      35  33.0
2     IN   32      69  23.0
3     CA   35      54  54.0
4     NY   29      78  11.0
5     TX   35      54  27.0
6     MD   34      67  81.0
7     IN   41      35  24.0
8     CA   29      78  10.0
9     NY   34      67   7.0
10    TX   34      65   NaN
11    TX   42      45   NaN
12    TX   23      46   NaN
13    TX   36      65   NaN
14    TX   42      53   NaN
15    TX   25      62   NaN
16    TX   35      66   NaN
17    TX   40      75   NaN
18    MD   27      22   NaN
19    MD   23      69   NaN
20    MD   25      73   NaN
21    MD   29      75   NaN
22    MD   24      65   NaN
23    MD   25      66   NaN
24    MD   31      78   NaN
25    MD   22      81   NaN
26    MD   25      73   NaN
27    MD   30      62   NaN
28    TX   32      54   NaN
29    MD   29      69   NaN
30    IN   41      35   NaN
31    CA   35      67   NaN
32    NY   34      78   NaN
Test passed 32


### Exercise 4 - Create a new Pop column in county_level_df

Looking at the tables country_level_df and state_level_df, each has a column containing data on population.  One column is headed *Population* and the other is headed *Pop*.  Ideally these would be merged into one column and this would minimise the number of NaN entries.  

Write a function called **create_pop** that:

*  Add a new column called '`Pop`' to the `county_level_df` dataframe which contains a copy of all the values in the '`Population`' column  (`df['new_name'] = df['existing_name']`)  
*  Drop the 'Population' column `df.drop([column name], axis=1)` from `county_level_df` and store the result in a new dataframe called **county_level_df_v2**



In [ ]:
def create_pop():
  # add code below to join the income_new dataframe with the income_df dataframe ignoring the index
  
  county_level_df["Pop"] = county_level_df["Population"]
  county_level_df_V2= county_level_df.drop(["Population"], axis=1)
  return county_level_df_V2


# save returned dataframe in a variable
county_level_df_v2 = create_pop()

# This will run and test your code to see if new dataframe contains pop column and dropped population column 
if 'Pop' in county_level_df_v2.columns and len(county_level_df_v2.columns) == 5:
  print("Test passed, contains 5 columns including Pop column")
elif 'Pop' in county_level_df_v2.columns and len(county_level_df_v2.columns) != 5:
  print("Test not passed, expected 5 columns, instead got", len(county_level_df_v2.columns))
else: 
  print("Test not passed, column Pop not present")


Test passed, contains 5 columns including Pop column


## Exercise 5 - clean up the format of the Pop column in state_level_df

Now that both dataframes hava a Pop column, we should make the data consisent in format for both dataframes.

In `state_level_df` the values in the `Pop` column are in 1000s.  In `county_level_df_v2` the values in the `Pop` column are actual numbers.  Let's convert the `state_level_df` values from 1000s for consistency.  We can do this by performing an operation on a column ( `df[column name] = df[column name] // 1000 `) 

first copy the state_level_df into a new variable called **state_level_df_copy** you can do this like: `df_new = df.copy()` 

Write a function called **clean_pop** which will:

*  convert the values in the `Pop` column of the `state_level_df` dataframe from numbers of 1000s to actual numbers, rounding to whole numbers



In [ ]:

state_level_df = pd.read_excel("https://github.com/futureCodersSE/working-with-data/blob/main/Data%20sets/Income-Data.xlsx?raw=true", sheet_name = 'state-level')
state_level_df_copy = state_level_df.copy()

def clean_pop():
  
  #add code below which converts the Pop column to actual numbers rather than 1000s 
  state_level_df_copy['Pop']=state_level_df['Pop']//1000
  return state_level_df_copy

  
  

# create new variable 
state_level_df_v2 = clean_pop()


# This will run and test your code to see if you've correctly converted the column to 10s instead of 1000s
actual = state_level_df_v2['Pop'].max()
expected = 29

if actual == expected and len(income_df_v2.columns) == 4:
  print("Test passed", actual)
else:
  print("Test failed, expected max of", expected, "got", actual)


Test passed 29


### Exercise 6 - combine the two v2 dataframes using concat

Write function **combine_v2** which will: 

Combine `county_level_df_v2` and `state_level_df_v2` to create a new dataframe called **income_df_v3**.  The join type should be 'inner' and ignore_index should be True


In [ ]:

def combine_v2():
  # return the 2 dataframes combined joined inner and ignoring index 
  income_df_v3= pd.concat([county_level_df_v2, state_level_df_v2], join="inner", ignore_index=True)#no axis
  return income_df_v3

# save within a new dataframe
  
income_df_v3=combine_v2()
  



# This will run and test your code to see if your new dataframe is correct length and has correct number of columns 

actual = len(income_df_v3)
actual2 = len(income_df_v3.columns)
expected = 23 
expected2 = 4 

if actual == expected and actual2 == expected2:
  print("Test passed", actual, "rows and 4 columns")
else:
  print("Test failed, expected", expected, "rows", expected2, "columns but got", actual, "rows and", actual2,"columns")

combine_v2()
print(combine_v2())

Test passed 23 rows and 4 columns
   State  Age  Income  Pop
0     TX   34      65   72
1     TX   42      45   33
2     TX   23      46   25
3     TX   36      65   54
4     TX   42      53   11
5     TX   25      62   28
6     TX   35      66   82
7     TX   40      75    5
8     MD   27      22   61
9     MD   23      69    5
10    MD   25      73   98
11    MD   29      75   64
12    MD   24      65   36
13    MD   25      66   24
14    MD   31      78   34
15    MD   22      81   89
16    MD   25      73   21
17    MD   30      62   21
18    TX   32      54   23
19    MD   29      69   10
20    IN   41      35    5
21    CA   35      67   29
22    NY   34      78   18


### Exercise 7 - Combining dataframes using merge 
---
read in 2 new dataframes called **skill_df** and **industry_df** using the 'Skill Migration' sheet and 'Industry Migration' sheet from the following excel spreadsheet:
"https://github.com/futureCodersSE/working-with-data/blob/main/Data%20sets/public_use-talent-migration.xlsx?raw=true"

Write a function called `get_combine()` which will: 

Use the merge() function to combine the industry and skill migration dataframes, merging on the country column and the skill group column (industry_name in industry sheet). Only keep the two merge columns and 'wb_income' and 'net_per_10k_2019' columns when merging to create a new dataframe called `migration_df`. 

In [ ]:
import pandas as pd
url="https://github.com/futureCodersSE/working-with-data/blob/main/Data%20sets/public_use-talent-migration.xlsx?raw=true"
skill_df=pd.read_excel(url, sheet_name="Skill Migration")
industry_df=pd.read_excel(url,sheet_name="Industry Migration")


def get_combine():
  # merge dataframes into variable called migration_df and return it 
  #single column pd.merge(df1, df2, left_on = 'df1_column', right_on = 'df2_column')
  #migration_df= pd.merge(skill_df, industry_df, left_on=['country_name', 'skill_group_name'], right_on=['country_name', 'industry_name'])#MY CODE
  migration_df=pd.merge (skill_df, industry_df, left_on=['country_name','skill_group_name'], right_on=['country_name','industry_name'])#Charlie's CODE

  

  #return migration_df[['industry_name','skill_group_name', 'wb_income_x', 'wb_income_y','net_per_10k_2019_x', 'net_per_10k_2019_y']]#MY CODE
  return migration_df[['industry_name','skill_group_name', 'wb_income_x', 'wb_income_y','net_per_10K_2019_x', 'net_per_10K_2019_y']]#Charlie's CODE

# This will run and test your code to see if your new dataframe is correct length and has correct number of columns 

actual = len(get_combine())
actual2 = len(get_combine().columns)
expected = 873 
expected2 = 6

if actual == expected and actual2 == expected2:
  print("Test passed", actual, "rows", actual2, "columns")
else:
  print("Test failed, expected", expected, "rows", expected2, "columns but got", actual, "rows and", actual2,"columns")

#get_combine()
#print(get_combine())


Test passed 873 rows 6 columns


### Exercise 8 - pivot table of state and population 
---

Write a function called **create_pivot** which will:

Create a pivot table of `Pop` by `State` and store the result in a new dataframe called **population_pivot**  using `income_df_v3` created in exercise 6

To make a pivot table:

`df_pivot = pd.pivot_table(
      df, 
      values = 'column1 name',
      index = 'column2 name', 
      columns = 'column3 name',
      aggfunc = np.mean
)`  
If index column is not specified, it will automatically use the existing dataframe index   
Make sure to `import numpy as np` if you want to aggregate the means using `aggfunc`. 

**Test Input**  
population_pivot.shape   
**Test Output**  
(1, 5)




In [ ]:
import numpy as np
import pandas as pd


def create_pivot(df):
  #add code below which creates a pivot table of Pop and State 
  population_pivot = pd.pivot_table( income_df_v3, values = 'Pop', index = 'State', aggfunc = np.mean )

  

  return population_pivot



#save series in a new variable
population_pivot = create_pivot(income_df_v3)


# This will run and test your code to see if your new series is the correct length
actual = len(population_pivot)
expected = 5 

if actual == expected:
  print("Test passed", actual)
else:
  print("Test failed expected", expected, "got", actual)

print()

Test passed 5


# Reflection
----

## What skills have you demonstrated in completing this notebook?

Your answer: 

## What caused you the most difficulty?

Your answer: 